# Imports

In [ ]:
import matplotlib.pyplot as plt
import optuna
import pandas as pd
import seaborn as sns

# from sklearn.linear_model import <model
from sklearn.model_selection import train_test_split

from src.preprocessing import preprocess_data
from src.utils.compute_metrics import compute_and_store_metrics
from src.utils.confusion_matrix import plot_confusion_matrix
from src.utils.load_data import load_data


# Load Data

In [ ]:
X, y = preprocess_data(load_data())

# Model

## Your Model

In [ ]:
## Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
## Model
model = ...
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[
    :, 1
]  # Probability estimates for the positive class (default)

In [ ]:
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

In [ ]:
plot_confusion_matrix(y_test, y_pred, cmap="Blues")

## Hyperparameter fine-tuning (Optuna)

In [ ]:
def objective(trial):
    # Suggest hyperparameters
    n_estimators = ...
    max_depth = ...
    min_samples_split = ...

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred_proba = model.predict_proba(X_test)[
        :, 1
    ]  # Probability estimates for the positive class (default)
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Calculate F1-score (or any other metric you prefer)
    from sklearn.metrics import f1_score

    score = f1_score(y_test, y_pred)

    return score


# Create and run the study
study = optuna.create_study(direction="maximize")  # Maximize F1-score
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n" + "=" * 60)
print("BEST HYPERPARAMETERS")
print("=" * 60)
print(f"Best F1-Score: {study.best_value:.4f}")
print(f"\nBest Parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

In [ ]:
plot_confusion_matrix(y_test, y_pred, cmap="Greens")

In [ ]:
compute_and_store_metrics(y_test, y_pred, model_name="Random Forest")

In [ ]:
save_model(model, model_name="random_forest")